In [12]:
from datasets import load_dataset
import numpy as np
import math

In [2]:
docs = load_dataset(f"Cohere/wikipedia-22-12-en-embeddings", split="train", streaming=True)

Resolving data files:   0%|          | 0/253 [00:00<?, ?it/s]

In [5]:
for doc in docs:
    print(len(doc["emb"]))
    break

768


In [17]:
EMBEDDING_DIM = 768
NUM_TREES = 10
NUM_EMBEDDINGS = 35_167_920

# Indicates target size for all embeddings in a leaf node.
# Treat this like a 'page size', in that the client will load it all at once.
TARGET_LEAF_SIZE_BYTES = 1024 * 1024
EMBEDDING_SIZE_BYTES = EMBEDDING_DIM * 8
TARGET_LEAF_NUM_EMBEDDINGS = TARGET_LEAF_SIZE_BYTES // EMBEDDING_SIZE_BYTES

TREE_DEPTH = math.ceil(math.log2(NUM_EMBEDDINGS) - math.log2(TARGET_LEAF_NUM_EMBEDDINGS))

In [24]:
random_vectors = np.random.normal(size=(NUM_TREES, TREE_DEPTH, EMBEDDING_DIM))

In [30]:
sample_emb = np.array(doc["emb"])

In [48]:
rand_dot_prods = random_vectors @ sample_emb